In [ ]:
import transformers
import peft
import pandas as pd
import datasets
import torch
import gc

model_name = 'tiiuae/falcon-7b'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device
gc.collect()

In [ ]:
def load_model(model_name: str):
    m = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map='cuda',
        trust_remote_code=False,
        revision='main',
    )

    return m.to(device)

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
m = load_model(model_name=model_name)

In [ ]:
def load_and_prepare_dataset(model_name: str):
    df = pd.read_csv('/content/data_final.csv')
    df = df[['text', 'is_major_principle_here', 'ethical_us']]
    transparency = "Transparency: Transparency can typically be understood in two ways: the transparency of the AI technology itself and the transparency of the AI organisations developing and using it. Throughout our analysis, transparency was regularly discussed directly, or in relation to processes required to ensure it, such as explainability, understandability and communication."
    n_malef = "Non-maleficence: The principle of non-maleficence gained attention, and in its most basic form, it means to do no harm or avoid doing harm to others"
    resp = "Responsibility: The principle of responsibility is base on responsibility of the developers and stakeholders over the AI, accountability, liability and acting with integrity"
    privacy = "Privacy: Related to privacy of personal data, because of the large abundance of data that is required for AI to work, it is important that individuals privacy is not jeopardised as a result"
    benefiecnce = "Beneficence: Beneficence essentially means to do good, to carry out an activity with the intention of benefitting someone or society as a whole"
    f_and_a = "Freedom and autonomy: Democratic societies place value in freedom and autonomy, and it is important that AI use does not encumber or harm these for us"
    sus = "Sustainability: All fields and disciplines are affected and need to incorporate sustainability agendas"
    dig = "Dignity: Human dignity is the recognition that individuals have inherent worth and that their rights should be respected"
    j_and_f = "Justice and fairness: The issue of discrimination and unfair results resulting from algorithms has become a significant concern. It is imperative that systems are designed to ensure that they are free from any form of unfairness and inequality."
    trust = "Trust: Trust is built by keeping promises, making sure systems work properly and protecting data responsibly. Organisations must prove their trustworthiness by ensuring that their technologies are secure and effective."
    instruction = f'You are an ethical requirements engineer translating requirements to ethical user stories based on one of the 10 ethical principles: {transparency}\n{n_malef}\n{resp}\n{privacy}\n{benefiecnce}\n{f_and_a}\n{sus}\n{dig}\n{j_and_f}\n{trust}\nand the TEMPLATE:\nAs a <persona> i want to <do something> <so that>\nNow following this template and the ethical principles defined above, choose only one of the ethical principles and transform the requirement below into a description in one line of an ethical user story, substituting <persona> for the one that requires this functionality, <do something> for what the <persona> wants to do and <so that> to the end goal of the functionality'

    # df['data'] = df.apply((lambda row: f'''### Human: {instruction}\n{row['text']}]\n ### Assistant: {row['ethical_us']}'''), axis=1)
    df['data'] = df.apply((lambda row: f'''<s>[INST] {instruction}\n{row['text']}\n[/INST] {row['ethical_us']}</s>'''), axis=1)

    t = transformers.AutoTokenizer.from_pretrained(model_name, use_fast=True)
    # t.add_special_tokens({'pad_token': '<PAD>'})
    dt = datasets.Dataset.from_pandas(df)
    dt = dt.train_test_split(test_size=0.2)

    t.pad_token = t.eos_token
    data_collator = transformers.DataCollatorForLanguageModeling(t, mlm=False)

    return dt, data_collator, t

data, data_collator, t = load_and_prepare_dataset(model_name=model_name)


In [ ]:
def tokenize_function(examples):
    # extract text
    text = examples["data"]

    #tokenize and truncate text
    t.truncation_side = "left"
    tokenized_inputs = t(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

tokenized_data = data.map(tokenize_function, batched=True)

In [ ]:
def train_model(model, lr, batch_size, num_epochs, tokenized_data, collator):
    model.train() # training state
    model.gradient_checkpointing_enable()
    model = peft.prepare_model_for_kbit_training(model) # turn into qlora

    # lora config
    config = peft.LoraConfig(
        r=32,
        lora_alpha=64,
        target_modules=["q_proj"],
        lora_dropout=.1,
        bias="none",
        task_type="CAUSAL_LM"
    )
    config.inference_mode = False

    model = peft.get_peft_model(model, config) # model in lora style

    training_args = transformers.TrainingArguments(
        output_dir= "checkpoints_output",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        warmup_steps=2,
        fp16=True,
        optim="paged_adamw_8bit",
    )

    trainer = transformers.Trainer(
        model=model,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["test"],
        args=training_args,
        data_collator=collator
    )

    # train model
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
    trainer.train()

    # renable warnings
    model.config.use_cache = True

    return model

final_model = train_model(m, 2e-4, 4, 3, tokenized_data, data_collator)

In [ ]:
prompt_template = lambda comment: f'''[INST] Transform the requirement below into a one line brief description of an ethical user story \n{comment} \n[/INST]'''
comment = "The user need an authentication token to login."
prompt = prompt_template(comment)

final_model.eval()
inputs = t(prompt, return_tensors="pt")
outputs = final_model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=52)
print(t.batch_decode(outputs)[0])